<a href="https://colab.research.google.com/github/alitarikarslanoglu/ClaimAnalysis/blob/main/claim_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Claim Analysis

## System Check and Instalation of Reqirements

In [ ]:
!nvidia-smi

Wed Mar 20 09:40:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### System Requirements

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.0 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off
!pip install -qqq selenium==4.14.0 --progress-bar off
!pip install -qqq bs4 --progress-bar off
!pip install -qqq tqdm --progress-bar off
!pip install -qqq PyPDF2 --progress-bar off
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb
!pip install -qqq  auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!apt install chromium-chromedriver
!sudo apt-get install poppler-utils

In [7]:
!apt-get install python3-pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,967 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip all 22.0.2+dfsg-1ubuntu0.4 [1,305 kB]
Fetched 1,677 kB in 1s (1,162 kB/s)
Selecting previously unselected package python3-setuptools.
(Reading database ... 122406 files and directories cur

### Imports

In [ ]:
#import
import torch
from tqdm import tqdm
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline
import pandas as pd
import re
import os
import datetime
import shutil
import glob
import time
from tqdm import tqdm
from google.colab import drive
from IPython.display import clear_output
from PyPDF2 import PdfReader
import random

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

ModuleNotFoundError: No module named 'auto_gptq'

## Retrival

### Selenium WebDriver for Chrome

In [ ]:
from selenium import webdriver

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument('ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    return driver

###Expert Report/CSV Retrival

In [ ]:
!mkdir data
!mkdir pdfs
!gdown 1Xgb7GwakWa_wEiOAeiFz-sUg8Uo0NO3F -O data/tree.xlsx
!rm -r pdfs

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘pdfs’: File exists
Downloading...
From: https://drive.google.com/uc?id=1Xgb7GwakWa_wEiOAeiFz-sUg8Uo0NO3F
To: /content/data/tree.xlsx
100% 541k/541k [00:00<00:00, 127MB/s]


In [ ]:
claims = pd.read_excel('data/tree.xlsx')

### List of EFSA Opinion References Links

In the google sheet created by experts, "EFSA Opinion Reference" feature contains the codes of the documents. Following code creates the usable links to those Scientific opinions
https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2009.1226

In [ ]:
res = []
for k in tqdm(range(len(claims['EFSA Opinion Reference'])), position=0, leave=True):
  i = claims['EFSA Opinion Reference'][k]
  claim = claims['Claim'][k]
  if str(i) == 'nan':
    continue
  row = re.split(',',str(i))
  for j in row:
    args = re.split(';|:',j)
    if len(args)!=3:
      continue
    res.append('https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.' + args[0].replace(" ","") + '.' + args[2].replace(" ","") + '#'+ claim + '#' + str(k))

  row = re.split('-',str(i))
  args = re.split('-',j)
  if len(args)!=3:
    continue
  res.append('https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.' + args[1].replace(" ","") + '.' + args[2].replace(" ","") + '#'+ claim + '#' + str(k))

In [ ]:
def getpage(url, c, out_subdir):
  log = ''
  try:
    for k in os.listdir('pdfs/'+out_subdir+'/'):
      n = re.split('-',k)
      if len(n)!=8:
        continue
      if (str(c) + '.pdf') == n[7]:
        log += f'\tFile n {c} skipped already present\tfilename:{k}\n'
        return log
  except:
    pass

  driver = driversetup()
  driver.get(url)
  driver.close()

  name = 'pdfs/'+ out_subdir + '/efsa-' + str(datetime.datetime.now().strftime("%Y-%m-%d-at-%H-%M-")) + str(c) + '.pdf'

  try:
    filename = max([f for f in os.listdir('.')],key=os.path.getctime)
    try:
      os.mkdir('pdfs/'+ out_subdir)
    except:
      pass
    shutil.move(filename,os.path.join('.',name))
  except:
    log += f'File n {c} failed reffer to the followring url.\n{url}\n'
  return log